# **Minicurso teórico-prático em Oncogenômica**

Neste minicurso abordaremos aplicações práticas da **oncogenômica**, destacando sua importância na pesquisa, no diagnóstico e no tratamento do câncer através dos seguintes tópicos:


*   Instalação de programas/ferramentas/bibliotecas

*   Download dos dados de sequenciamento e genoma de referência

*   Análise de qualidade de um sequenciamento

*   Alinhamento das leituras de sequenciamento contra um genoma de referência

*   Chamada de variantes

*   Anotação de variantes e interpretação de resultados


Use Ctrl+Enter para executar o conteúdo de uma 'célula' de código

Use Shift+Enter para executar código selecionado pelo mouse

Para começar, vamos instalar algumas bibliotecas e pacotes que serão usados ao longo do minicurso

# Instalando bibliotecas e o gerenciador mamba através do PIP



Aqui será instalado o Mamba, uma ferramenta de gerenciamento de pacotes e ambientes, semelhante ao Conda, mas com algumas melhorias de desempenho. O Mamba será instalado no ambiente Colab através do PIP, que é o gerenciador de pacotes oficial para Python, utilizado para instalar e gerenciar bibliotecas e pacotes de software escritos nessa linguagem.

Para mais informações nos links abaixo:

* [Conda](https://docs.conda.io/en/latest/)
* [Mamba](https://mamba.readthedocs.io/en/latest/user_guide/mamba.html)
* [PIP](https://pip.pypa.io)

In [ ]:
#Usando instalador de bibliotecas do Python (pip) para instalar o conda/mamba.
# Conda/Mamba é uma ferramenta que permite o gerenciamento de pacotes e ambientes.
# Que é um espaço isolado onde você pode instalar e executar ferramentas específicas para um projeto
!pip install --upgrade --force-reinstall zstandard
!pip install -q condacolab

!sed -i '/cudatoolkit/d' /usr/local/conda-meta/pinned

#Código em Python
import condacolab
condacolab.install()

#Instala algumas dependências necessárias para os próximos passos
#conda-forge é um canal comunitário em que ficam ferramentas e pacotes
#bioconda é um canal especializado em bioinfomática
!mamba install -c conda-forge curl --quiet


#Lista os softwares do ambiente atual do conda/mamba
!mamba env list

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 52.4 MB/s eta 0:00:00
sed: can't read /usr/local/conda-meta/pinned: No such file or directory
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
# conda environments:
#
base                     /usr/local



#Baixando o genoma de referência e sequências de interesse



Entrez é uma ferramenta de busca e recuperação de informações biológicas oferecida pelo NCBI (National Center for Biotechnology Information). Ele é uma interface integrada que permite o acesso a diversos bancos de dados do NCBI, com sequências de DNA e proteínas, artigos científicos, genomas, estruturas 3D, variações genéticas e muito mais.

Abaixo criaremos outro ambiente Mamba para a instalação do Entrez.

Mais informações sobre o Entrez:
* [ENTREZ](https://www.ncbi.nlm.nih.gov/search/)
* [Manual Entrez](https://www.ncbi.nlm.nih.gov/books/NBK3837/)

In [ ]:
#Criando ambiente para baixar o genoma de referência e anotações
!mamba create -n entrez_ncbi
!mamba env list
!mamba install -c bioconda cmake entrez-direct --quiet
!mamba activate entrez_ncbi


Looking for: []

Preparing transaction: - done
Verifying transaction: | / - done
Executing transaction: | done

To activate this environment, use

     $ mamba activate entrez_ncbi

To deactivate an active environment, use

     $ mamba deactivate

# conda environments:
#
base                     /usr/local
entrez_ncbi              /usr/local/envs/entrez_ncbi

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.



Vamos criar a pasta "refGen" e baixar o genoma de referência.

In [ ]:
# O comando mkdir em shell é utilizado para criar pastas. Já o comando pwd mostra o caminho do diretório local enquanto o comando ls lista arquivos e diretórios
!echo -e "### Criando a pasta dados\n"
!mkdir refGen
!echo -e "### O diretório atual é: $PWD \n"
!echo -e "### O conteúdo de \"$PWD\" é:\n"
!ls -lh

### Criando a pasta dados

### O diretório atual é: /content 

### O conteúdo de "/content" é:

total 32K
-rw-r--r-- 1 root root  23K Nov  6 11:57 condacolab_install.log
drwxr-xr-x 2 root root 4.0K Nov  6 12:11 refGen
drwxr-xr-x 1 root root 4.0K Nov  4 14:22 sample_data


Agora vamos baixar o cromossomo 17 do genoma de referência GRCh38. Ele será usado como genoma de referência nas análises seguintes.

In [ ]:
!echo -e "### Baixando o cromossomo 17\n"
!esearch -db nucleotide -query "NC_000017.11" | efetch -format fasta > ./refGen/chr17_GRCh38.fasta
!echo -e "### Verificando o arquivo baixado\n"
!ls -lh ./refGen

### Baixando o cromossomo 17

### Verificando o arquivo baixado

total 81M
-rw-r--r-- 1 root root 81M Nov  6 12:20 chr17_GRCh38.fasta


Vamos baixar também nossos arquivos com os dados de sequenciamento e uma minibase de dados baseada no Clinvar que será usada para a anotação de variantes. Aqui baixaremos 3 dados de sequenciamento, relativos ao gene BRCA1 normal e duas versões do mesmo gene contendo mutações conhecidas.

* Mais informações do o gene [BRCA1](https://www.cancer.gov/about-cancer/causes-prevention/genetics/brca-fact-sheet)

In [ ]:
#TESTE
!echo -e "\n### Baixando as sequências de interesse do github para a pasta dados"
!git --version
!git clone https://github.com/jlpitta/sbbs2024.git ./brca1




### Baixando as sequências de interesse do github para a pasta dados
git version 2.34.1
Cloning into './brca1'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 4), reused 14 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 7.00 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (4/4), done.


#Análise de qualidade de dados em sequenciamento Illumina - formato Fastq

Todos os sequenciadores produzem dados em um formato chamado **fastq**. A estrutura é mostrada abaixo. Todas as sequências com fastq são representadas por 4 linhas:

```
@SEQ_ID                   <---- Identificador da sequência
AGCGTGTACTGTGCATGTCGATG   <---- Sequência de nucleotídeos
+                         <---- Separador
%%).1***-+*''))**55CCFF   <---- ASCII - caracteres que representam a qualidade (phred score)

```

A qualidade das sequências é representada como um caractere do código ASCII (informação sobre o que é ASCII [aqui](https://pt.wikipedia.org/wiki/ASCII)). Verifique o significado de qualidade dos código [aqui](https://help.basespace.illumina.com/files-used-by-basespace/quality-scores) para obter uma explicação .
Os valores numéricos correspondem aos valores de qualidade phred

10, 1 em 10	- 90%

20, 1 em 100	- 99%

30, 1 em 1000	- 99.9%

40, 1 em 10,000	- 99.99%

50, 1 em 100,000	- 99.999%

60, 1 em 1,000,000	- 99.9999%

Vamos olhar a estrutura dos nossos arquivos de sequenciamento em fastq

In [ ]:
!echo -e "### Abrindo o começo do arquivo BRCA1_WT_R1.fq.gz\n"
!zcat ./brca1/BRCA1_WT_R1.fq.gz | head

### Abrindo o começo do arquivo BRCA1_WT_R1.fq.gz

@NG_005905.2-25820/1
TTTAGCTTTATTCTGGTCTTTTTAATTTTCTTTTTTTTTTTCAGACAGAGTCTCGTTCTGTCGCCCAGGCTGGAGTGCAGTGGCACCATCTCGGCTCTCTGTAACCTCCGCCTCCTGAATTCAAGTGATTCTCCTGCCTCAGCCTCCCGA
+
CCCGGGGGGGGGGJJGJ=JGJGJJCJJJJJCGJJ8GJJGJGJJJJJGJJGJJGJJGJGJGJJGJGGCGGG=JGJGCG(J=CG=GGGGG=GG=GGGGGGGGJ8GGGGGGGGGGGGGGGGGGGGGC=GGGGGGGGCGG8GCGGGGGGCCGGG
@NG_005905.2-25818/1
ACCTCTTCCTTCACCAACTGTACTCAACACCCTTCTGTAAACAGAGTGAGAACCTGGGTTCATGGACAAGCTCTTTTCCACTTATCTTCGGGTTAAACCAAAACTTTTTCAGCAACTTTGCCCCTGTCCAAGTTTTGCAGAACACCAGCT
+
1CCGGGC=GGGGGJJJJJ(JGJJ1JJJJJJ(JJJGJGGJC=GGC8GJGCJGGGGGGGJCJGCGGGGGGGGGGGCJGGGCGCGGGCC8GGCCCCG=G1GGGJCGCGGCGCGGGCG=GGCGGC=GGGGCGGGG=C8CGGGGGGCC1GGGGGC
@NG_005905.2-25816/1
GCTTTCTCTTTCTTGGAGAAAGGAAAAGACCCAAGGGGTTGGCAGCAATATGTGAAAAAATTCAGAATTTATGTTGTCTAATTACAAAAAGCAACTTCTAGAATCTTTAAAAATAAAGGACGTTGTCATTAGTTCTTTGGTTTGTATTAT


Agora precisamos de uma forma mais simples de fazer a análise de qualidade, pois olhando diretamente o arquivos fastq não é muito intuitivo. Para isso, vamos instalar e usar o FastQC.

Para mais informações sobre o FastQC:

* [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)

In [ ]:
#Criando ambientes para o controle de qualidade
!mamba create -n quality
!mamba env list
!mamba activate quality
!mamba install -c bioconda fastqc multiqc --quiet


Looking for: []

Preparing transaction: - done
Verifying transaction: | / - \ | / - done
Executing transaction: | done

To activate this environment, use

     $ mamba activate quality

To deactivate an active environment, use

     $ mamba deactivate

# conda environments:
#
base                     /usr/local
entrez_ncbi              /usr/local/envs/entrez_ncbi
quality                  /usr/local/envs/quality

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [ ]:
#Executando o FastQC para gerar relatórios de qualidade das sequências
!echo -e "### Listando arquivos na pasta brca1 \n"
!ls ./brca1/
!echo -e "\nCriando a pasta fastqc para armazenar os resultados do fastqc\n"
!mkdir fastqc
!echo -e "\n### Rodando o FastQC na para os arquivos da pasta brca1 \n"
!fastqc ./brca1/*.fq.gz -o ./fastqc
!echo -e "\n### Listando novamente arquivos na pasta brca1 \n"
!ls ./fastqc

### Listando arquivos na pasta brca1 

BRCA1_185delAG_R1.fq.gz    BRCA1_c.5266dupC_R2.fq.gz  clinvar_mini.vcf.gz
BRCA1_185delAG_R2.fq.gz    BRCA1_WT_R1.fq.gz	      clinvar_mini.vcf.gz.tbi
BRCA1_c.5266dupC_R1.fq.gz  BRCA1_WT_R2.fq.gz	      README.md

Criando a pasta fastqc para armazenar os resultados do fastqc


### Rodando o FastQC na para os arquivos da pasta brca1 

application/gzip
application/gzip
Started analysis of BRCA1_185delAG_R1.fq.gz
application/gzip
application/gzip
application/gzip
application/gzip
Approx 5% complete for BRCA1_185delAG_R1.fq.gz
Approx 15% complete for BRCA1_185delAG_R1.fq.gz
Approx 20% complete for BRCA1_185delAG_R1.fq.gz
Approx 30% complete for BRCA1_185delAG_R1.fq.gz
Approx 35% complete for BRCA1_185delAG_R1.fq.gz
Approx 45% complete for BRCA1_185delAG_R1.fq.gz
Approx 50% complete for BRCA1_185delAG_R1.fq.gz
Approx 60% complete for BRCA1_185delAG_R1.fq.gz
Approx 65% complete for BRCA1_185delAG_R1.fq.gz
Approx 75% complete for BRCA1_185delAG_R1.fq.gz
App

Vamos visualizar a análise do FastQC baixando o arquivo html gerado.

Após a análise da qualidade, vamos fazer uma filtragem de sequências de baixa qualidade, visando um dado de melhor qualidade para as análises seguintes. Para isso, instalaremos o fastp.

Para mais informações:

* [fastp](https://github.com/OpenGene/fastp)

In [ ]:
# Usando o mesmo ambiente conda do FastQC para instalar o fastp
!echo -e "\n### Instalando o fastp \n"
!mamba activate quality
!mamba install -c bioconda fastp --quiet


### Instalando o fastp 

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Agora usaremos o fastp recém instalado para fazer a filtragem das sequências

In [ ]:
# Utilizando o fastp
# É comum se referir a esta etapa como "trimagem" e esses arquivos como "trimados"
!echo -e "\n### Criando a pasta trimmed para armazenar os arquivos trimados \n"
!mkdir trimmed
!echo -e "\n### Filtrando as sequências com o fastp \n"

!fastp -i ./brca1/BRCA1_WT_R1.fq.gz -I ./brca1/BRCA1_WT_R2.fq.gz --detect_adapter_for_pe -o ./trimmed/BRCA1_WT_R1_Trim.fq.gz -O ./trimmed/BRCA1_WT_R2_Trim.fq.gz -h ./trimmed/BRCA1_WT_Trim.html --qualified_quality_phred 20
!fastp -i ./brca1/BRCA1_185delAG_R1.fq.gz -I ./brca1/BRCA1_185delAG_R2.fq.gz --detect_adapter_for_pe -o ./trimmed/BRCA1_185delAG_R1_Trim.fq.gz -O ./trimmed/BRCA1_185delAG_R2_Trim.fq.gz -h ./trimmed/BRCA1_185delAG_Trim.html --qualified_quality_phred 20
!fastp -i ./brca1/BRCA1_c.5266dupC_R1.fq.gz -I ./brca1/BRCA1_c.5266dupC_R2.fq.gz --detect_adapter_for_pe -o ./trimmed/BRCA1_c.5266dupC_R1_Trim.fq.gz -O ./trimmed/BRCA1_c.5266dupC_R2_Trim.fq.gz -h ./trimmed/BRCA1_c.5266dupC_Trim.html --qualified_quality_phred 20

!echo -e "Filtrando novamente o  BRCA1_WT mas aumentando o phred score para 35"
!fastp -i ./brca1/BRCA1_WT_R1.fq.gz -I ./brca1/BRCA1_WT_R2.fq.gz --detect_adapter_for_pe -o ./trimmed/BRCA1_WT_R1_Trim35.fq.gz -O ./trimmed/BRCA1_WT_R2_Trim35.fq.gz -h ./trimmed/BRCA1_WT_Trim35.html --qualified_quality_phred 35



### Criando a pasta trimmed para armazenar os arquivos trimados 


### Filtrando as sequências com o fastp 

Detecting adapter sequence for read1...
No adapter detected for read1

Detecting adapter sequence for read2...
No adapter detected for read2

Read1 before filtering:
total reads: 12910
total bases: 1936500
Q20 bases: 1898131(98.0186%)
Q30 bases: 1775298(91.6756%)

Read2 before filtering:
total reads: 12910
total bases: 1936500
Q20 bases: 1884320(97.3054%)
Q30 bases: 1735264(89.6083%)

Read1 after filtering:
total reads: 12910
total bases: 1936500
Q20 bases: 1898131(98.0186%)
Q30 bases: 1775298(91.6756%)

Read2 after filtering:
total reads: 12910
total bases: 1936500
Q20 bases: 1884320(97.3054%)
Q30 bases: 1735264(89.6083%)

Filtering result:
reads passed filter: 25820
reads failed due to low quality: 0
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate: 0.0154919%

Insert size peak 

Visualizando resultados

In [ ]:
#Visualizando o relatório gerado pelo fastp
!pwd
import IPython
#IPython.display.HTML(open('./trimmed/ERR5761182.fastp.html').read())
IPython.display.HTML('./trimmed/BRCA1_WT_Trim35.html')

/content


fastp version:,0.23.4 (https://github.com/OpenGene/fastp)
sequencing:,paired end (150 cycles + 150 cycles)
mean length before filtering:,"150bp, 150bp"
mean length after filtering:,"150bp, 150bp"
duplication rate:,0.015492%
Insert size peak:,199
total reads:,25.820000 K
total bases:,3.873000 M
Q20 bases:,3.782451 M (97.662045%)
Q30 bases:,3.510562 M (90.641931%)
GC content:,45.115931%


#Alinhamento das leituras de sequenciamento contra um genoma de referência

Vamos Iniciar instalando BWA, umas das principais ferramentas de montagem de genoma por referência.

Para mais informações sobre o BWA:

* [BWA](https://github.com/lh3/bwa)

In [ ]:
# Criando ambiente para bwa
!mamba create -n bwa
!mamba activate bwa
!mamba install -c bioconda bwa --quiet


Looking for: []

Preparing transaction: - done
Verifying transaction: | / - \ | / - \ | / - \ | / - \ done
Executing transaction: / done

To activate this environment, use

     $ mamba activate bwa

To deactivate an active environment, use

     $ mamba deactivate

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Agora vamos criar um índice para o genoma de referência

In [ ]:
#Criando arquivo índice
!bwa index -p ./refGen/chr17_GRCh38 ./refGen/chr17_GRCh38.fasta
# Criando pasta para colocar os arquivos pós-alinhamento
!mkdir aligned

[bwa_index] Pack FASTA... 0.76 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=166514882, availableWord=23716276
[BWTIncConstructFromPacked] 10 iterations done. 39120754 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 72271554 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 101732034 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 127912546 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 151177842 characters processed.
[bwt_gen] Finished constructing BWT in 58 iterations.
[bwa_index] 75.88 seconds elapse.
[bwa_index] Update BWT... 0.70 sec
[bwa_index] Pack forward-only FASTA... 0.51 sec
[bwa_index] Construct SA from BWT and Occ... 36.67 sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa index -p ./refGen/chr17_GRCh38 ./refGen/chr17_GRCh38.fasta
[main] Real time: 117.653 sec; CPU: 114.539 sec


Com a referência indexada, vamos alinhas as sequências de interesse contra a referência.

In [ ]:
# Alinhando as leituras trimadas contra a referência, gerando o arquivo de alinhamento .bam
!echo -e "Alinhando os genes BRCA1 mutados e não mutado contra a referência\n"
!bwa mem -R "@RG\tID:BRCA1_WT\tSM:BRCA1_WT\tPL:ILLUMINA" ./refGen/chr17_GRCh38 ./trimmed/BRCA1_WT_R1_Trim.fq.gz ./trimmed/BRCA1_WT_R2_Trim.fq.gz -o ./aligned/BRCA1_WT.bam
!bwa mem -R "@RG\tID:BRCA1_185delAG\tSM:BRCA1_185delAG\tPL:ILLUMINA" ./refGen/chr17_GRCh38 ./trimmed/BRCA1_185delAG_R1_Trim.fq.gz ./trimmed/BRCA1_185delAG_R2_Trim.fq.gz -o ./aligned/BRCA1_185delAG.bam
!bwa mem -R "@RG\tID:BRCA1_c.5266dupC\tSM:BRCA1_c.5266dupC\tPL:ILLUMINA" ./refGen/chr17_GRCh38 ./trimmed/BRCA1_c.5266dupC_R1_Trim.fq.gz ./trimmed/BRCA1_c.5266dupC_R2_Trim.fq.gz -o ./aligned/BRCA1_c.5266dupC.bam

Alinhando os genes BRCA1 mutados e não mutado contra a referência

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 25820 sequences (3873000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 12460, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (192, 198, 205)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (166, 231)
[M::mem_pestat] mean and std.dev: (198.55, 9.89)
[M::mem_pestat] low and high boundaries for proper pairs: (153, 244)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 25820 reads in 7.832 CPU sec, 7.948 real sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa mem -R @RG\tID:BRCA1_WT\tSM:BRCA1_WT\tPL:ILLUMINA -o ./aligned/BRCA1_WT.bam ./refGen/chr17_GRCh38 .

Para tratar os arquivos de alinhamento, instalaremos o Samtools, uma ferramenta amplamente usada para manipulação e análise de dados de sequenciamento de DNA.

Para mais informações sobre o Samtools:

* [Github/Samtools](https://github.com/samtools/samtools)
* [Manual](https://www.htslib.org/doc/samtools.html)

In [ ]:
# Criando ambiente para samtools e ivar para analisar e tratar os dados de alinhamento
!mamba create -n samtools
!mamba activate samtools
!mamba install -c bioconda samtools --quiet


Looking for: []

Preparing transaction: - done
Verifying transaction: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
Executing transaction: / done

To activate this environment, use

     $ mamba activate samtools

To deactivate an active environment, use

     $ mamba deactivate

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Vamos criar um índice da referência e ordenar as reads das sequências de interesse de acordo com a posição no genoma de referência usando o Samtools



In [ ]:
!echo -e "\nCriando índice para a referência\n"
# Cria aquivo índice baseado no arquivo FASTA da referência
!samtools faidx ./refGen/chr17_GRCh38.fasta

!echo -e "\nOrdenando leituras para BRCA1_WT\n"
# Ordena as leituras de acordo com a posição no genoma
!samtools sort -o ./aligned/BRCA1_WT_sorted.bam ./aligned/BRCA1_WT.bam
# Indexa arquivos SAM, BAM ou CRAN compactados
!samtools index ./aligned/BRCA1_WT_sorted.bam
# Faz uma avaliação completa do arquivo para gerar estatísticas básicas
!samtools flagstat ./aligned/BRCA1_WT_sorted.bam

!echo -e "\nOrdenando leituras para BRCA1_185delAG\n"
# Ordena as leituras de acordo com a posição no genoma
!samtools sort -o ./aligned/BRCA1_185delAG_sorted.bam ./aligned/BRCA1_185delAG.bam
# Indexa arquivos SAM, BAM ou CRAN compactados
!samtools index ./aligned/BRCA1_185delAG_sorted.bam
# Faz uma avaliação completa do arquivo para gerar estatísticas básicas
!samtools flagstat ./aligned/BRCA1_185delAG_sorted.bam

!echo -e "\nOrdenando leituras para BRCA1_c.5266dupC\n"
# Ordena as leituras de acordo com a posição no genoma
!samtools sort -o ./aligned/BRCA1_c.5266dupC_sorted.bam ./aligned/BRCA1_c.5266dupC.bam
# Indexa arquivos SAM, BAM ou CRAN compactados
!samtools index ./aligned/BRCA1_c.5266dupC_sorted.bam
# Faz uma avaliação completa do arquivo para gerar estatísticas básicas
!samtools flagstat ./aligned/BRCA1_c.5266dupC_sorted.bam


Criando índice para a referência


Ordenando leituras para BRCA1_WT

25820 + 0 in total (QC-passed reads + QC-failed reads)
25820 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
25820 + 0 mapped (100.00% : N/A)
25820 + 0 primary mapped (100.00% : N/A)
25820 + 0 paired in sequencing
12910 + 0 read1
12910 + 0 read2
25818 + 0 properly paired (99.99% : N/A)
25820 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)

Ordenando leituras para BRCA1_185delAG

25820 + 0 in total (QC-passed reads + QC-failed reads)
25820 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
25820 + 0 mapped (100.00% : N/A)
25820 + 0 primary mapped (100.00% : N/A)
25820 + 0 paired in sequencing
12910 + 0 read1
12910 + 0 read2
25818 + 0 properly paired (99.99% : N/A)
25820 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)

Verificando um dos arquivos gerados pelo Samtools

In [ ]:
# Imprime as primeiras leituras do arquivo BAM
!samtools view ./aligned/BRCA1_WT_sorted.bam | head

NG_005905.2-190	99	NC_000017.11	43024322	60	150M	=	43024373	201	TAGCTGGGATTACAGGCGCCCACCGTGCGGGTTGCACCGTGTTAGCCAGGATGATAGTCTCGATCTCCTGACCTCTTGATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGTATTACAGGCGTGAGCCACCGCGCCCGGCCTATTGTCAGAC	CC=1GGGGGGGGG=GJCGGGJGJJGGJGJJJGJ(GJJCJGGGGGGJGGJJJGJG1GJJGGGGJ8GGGJGGCGJGCCCGGGGCCGGCGGC=CGGCGGG=GGJGGGG(GGGGCCGGGGGCCGCGCGGGGCGGCGGGCCGGGGGCGCGCGGGG	NM:i:1	MD:Z:33T116	MC:Z:150M	MQ:i:60	AS:i:145	XS:i:78	RG:Z:BRCA1_WT
NG_005905.2-12378	99	NC_000017.11	43024345	60	150M	=	43024387	192	CGTGCGGGTTTCACCGTGTTAGCCAGGATGATAGTCTCGATCTCCTGACCTCTTGATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGTATTACAGGCGTGAGCCACCGCGCCCGGCCTATTGTCAGACTCTTAATCAAGCCTGACTTTTGG	CCCGGGGGGGGGGJJGJCJJJJJJGGJJJJJJJJGJJJGGGGJJGCJJGJG8CGJGGCJGCGJGC=CGGGGGGGG=GGGG=CGCGGG=GGGGGCGGCGGGCCCCGGCGGGGGGGGGC=GGGGGC=G1CCGCGGGGGGGG=GGGGGCGGCC	NM:i:0	MD:Z:150	MC:Z:150M	MQ:i:60	AS:i:150	XS:i:78	RG:Z:BRCA1_WT
NG_005905.2-4660	99	NC_000017.11	43024358	60	150M	=	43024404	196	CCGTGTTAGCCAGGATGATAGTCTCGATCTCCTGACCTCTTGATCCGCCCGCCTCGGCCTCCCAAAGT

Salvando somente as leituras que mapearam contra a referência e pegando o genoma montado por referência.

In [ ]:
!echo -e "\nCriando a pasta mapped para salvar os fastas mapeados\n"
!mkdir mapped

!echo -e "\nSalvando somente as leituras que mapearam contra a referência\n"
# É posossível salvar somente as leituras que mapearam contra o genoma
!samtools view -b -F 4 ./aligned/BRCA1_WT_sorted.bam > ./aligned/BRCA1_WT_mapped.bam
!samtools fasta ./aligned/BRCA1_WT_mapped.bam > ./mapped/BRCA1_WT_mapped_reads.fasta

!samtools view -b -F 4 ./aligned/BRCA1_185delAG_sorted.bam > ./aligned/BRCA1_185delAG_mapped.bam
!samtools fasta ./aligned/BRCA1_185delAG_mapped.bam > ./mapped/BRCA1_185delAG_mapped_reads.fasta

!samtools view -b -F 4 ./aligned/BRCA1_c.5266dupC_sorted.bam > ./aligned/BRCA1_c.5266dupC_mapped.bam
!samtools fasta ./aligned/BRCA1_c.5266dupC_mapped.bam > ./mapped/BRCA1_c.5266dupC_mapped_reads.fasta

# Imprime as primeiras linhas do arquivo
!echo -e "\nlistando o começo do arquivo BRCA1_WT_mapped_reads.fasta\n"
!head ./mapped/BRCA1_WT_mapped_reads.fasta

!echo -e "Listando os arquivos na pasta mapped\n"
!ls -lh ./mapped


Criando a pasta mapped para salvar os fastas mapeados


Salvando somente as leituras que mapearam contra a referência

[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 25820 reads
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 25820 reads
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 25820 reads

listando o começo do arquivo BRCA1_WT_mapped_reads.fasta

>NG_005905.2-190/1
TAGCTGGGATTACAGGCGCCCACCGTGCGGGTTGCACCGTGTTAGCCAGGATGATAGTCTCGATCTCCTGACCTCTTGATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGTATTACAGGCGTGAGCCACCGCGCCCGGCCTATTGTCAGAC
>NG_005905.2-12378/1
CGTGCGGGTTTCACCGTGTTAGCCAGGATGATAGTCTCGATCTCCTGACCTCTTGATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGTATTACAGGCGTGAGCCACCGCGCCCGGCCTATTGTCAGACTCTTAATCAAGCCTGACTTTTGG
>NG_005905.2-4660/1
CCGTGTTAGCCAGGATGATAGTCTCGATCTCCTGACCTCTTGATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGTATTACAGGCGTGAGCCACCGCGCCCGGCCTATTGTCAGACTCTTAATCAAGCCTGACTTTTGGGTTTAGCTCTGGC
>NG_005905.2-190/2
AGTTGCTCTGCTGGAGCCAGAGCTAAAC

In [ ]:
# Indexando os arquivos mapped
!echo -e "\nIndexando os arquivos mapped\n"
!samtools index /content/./aligned/BRCA1_WT_mapped.bam
!samtools index /content/./aligned/BRCA1_185delAG_mapped.bam
!samtools index /content/./aligned/BRCA1_c.5266dupC_mapped.bam


Indexando os arquivos mapped



In [ ]:
# Verificando arquivos
!echo -e "\nVerificando arquivo BRCA1_WT_mapped.bam\n"
!samtools view -H ./aligned/BRCA1_WT_mapped.bam
!echo -e "\nVerificando arquivo BRCA1_185delAG_mapped.bam\n"
!samtools view -H ./aligned/BRCA1_185delAG_mapped.bam
!echo -e "\nVerificando arquivo BRCA1_c.5266dupC_mapped.bam\n"
!samtools view -H ./aligned/BRCA1_c.5266dupC_mapped.bam


Verificando arquivo BRCA1_WT_mapped.bam

@HD	VN:1.5	SO:coordinate
@SQ	SN:NC_000017.11	LN:83257441
@RG	ID:BRCA1_WT	SM:BRCA1_WT	PL:ILLUMINA
@PG	ID:bwa	PN:bwa	VN:0.7.18-r1243-dirty	CL:bwa mem -R @RG\tID:BRCA1_WT\tSM:BRCA1_WT\tPL:ILLUMINA ./refGen/chr17_GRCh38 ./trimmed/BRCA1_WT_R1_Trim.fq.gz ./trimmed/BRCA1_WT_R2_Trim.fq.gz -o ./aligned/BRCA1_WT.bam
@PG	ID:samtools	PN:samtools	PP:bwa	VN:1.21	CL:samtools sort -o ./aligned/BRCA1_WT_sorted.bam ./aligned/BRCA1_WT.bam
@PG	ID:samtools.1	PN:samtools	PP:samtools	VN:1.21	CL:samtools view -b -F 4 ./aligned/BRCA1_WT_sorted.bam
@PG	ID:samtools.2	PN:samtools	PP:samtools.1	VN:1.21	CL:samtools view -H ./aligned/BRCA1_WT_mapped.bam

Verificando arquivo BRCA1_185delAG_mapped.bam

@HD	VN:1.5	SO:coordinate
@SQ	SN:NC_000017.11	LN:83257441
@RG	ID:BRCA1_185delAG	SM:BRCA1_185delAG	PL:ILLUMINA
@PG	ID:bwa	PN:bwa	VN:0.7.18-r1243-dirty	CL:bwa mem -R @RG\tID:BRCA1_185delAG\tSM:BRCA1_185delAG\tPL:ILLUMINA ./refGen/chr17_GRCh38 ./trimmed/BRCA1_185delAG_R1_Trim.fq.gz 

#Chamada de Variantes

Para a chamada de variantes usaremos o GATK, com a função HaplotypeCaller, usando o genoma de referência e as sequências de interesse previamente alinhadas contra o genômade referência.

Para mais informações sobre o GATK, clique nos links abaixo:

* [GATK/Broad Institute](https://gatk.broadinstitute.org/hc/en-us)
* [GATK - Documentação](https://gatk.broadinstitute.org/hc/en-us/categories/360002310591)




Primeiro, vamos instalar o GATK  

In [ ]:
# Criando ambiente mamba para o GATK e instalando a aplicação
!mamba create -n gatk
!mamba activate gatk
!mamba install -c bioconda gatk4 --quiet


Looking for: []

Preparing transaction: - done
Verifying transaction: | / - \ | / - \ | / - \ | / - \ | / - \ | done
Executing transaction: - done

To activate this environment, use

     $ mamba activate gatk

To deactivate an active environment, use

     $ mamba deactivate

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Agora criaremos um arquivo de índice de dicionário que contém informações sobre as referências genômicas usadas em uma análise. Ele é necessário para várias ferramentas do GATK, que requerem a ordenação e a estruturação correta da sequência de referência com a qual estão sendo comparadas as variantes genômicas.

In [ ]:
# Criando o dicionário para a referência
!gatk CreateSequenceDictionary \
    -R /content/refGen/chr17_GRCh38.fasta \
    -O /content/refGen/chr17_GRCh38.dict


Using GATK jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar CreateSequenceDictionary -R /content/refGen/chr17_GRCh38.fasta -O /content/refGen/chr17_GRCh38.dict
14:14:02.221 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Nov 06 14:14:02 UTC 2024] CreateSequenceDictionary --OUTPUT /content/refGen/chr17_GRCh38.dict --REFERENCE /content/refGen/chr17_GRCh38.fasta --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --versi

Aqui Faremos as chamadas de variantes para as 3 amostras que temos contra a referência. Demora em torno de 5 minutos.

In [ ]:
# Criando a pasta variants
!mamba activate gatk
!echo -e "\nCriando a pasta varients\n"
!mkdir variants
!echo -e "\nChamando variantes para BRCA1_WT\n"
!pwd
#!touch ./variants/BRCA1_WT.g.vcf.gz
!gatk HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_WT_mapped.bam -O ./variants/BRCA1_WT.g.vcf.gz -ERC GVCF
!gatk HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_185delAG_mapped.bam -O ./variants/BRCA1_185delAG.g.vcf.gz -ERC GVCF
!gatk HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_c.5266dupC_mapped.bam -O ./variants/BRCA1_c.5266dupC.g.vcf.gz -ERC GVCF

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.


Criando a pasta varients


Chamando variantes para BRCA1_WT

/content
Using GATK jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_WT_mapped.bam -O ./variants/BRCA1_WT.g.vcf.gz -ERC GVCF
14:14:50.695 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
SLF4J(W): Class path contains multiple SLF4J providers.
SLF4J(W): Found provider [org.apache.logging.slf4j.SLF4JServiceProvider@aa1bb14]
SLF4J(W): Found pro

Abaixo combinaremos as chamadas de variantes em um cohort.

In [ ]:
# Combinar os GVCF em um arquivo
!gatk CombineGVCFs -R /content/refGen/chr17_GRCh38.fasta -V ./variants/BRCA1_WT.g.vcf.gz -V ./variants/BRCA1_185delAG.g.vcf.gz -V ./variants/BRCA1_c.5266dupC.g.vcf.gz -O ./variants/cohort.g.vcf.gz

Using GATK jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar CombineGVCFs -R /content/refGen/chr17_GRCh38.fasta -V ./variants/BRCA1_WT.g.vcf.gz -V ./variants/BRCA1_185delAG.g.vcf.gz -V ./variants/BRCA1_c.5266dupC.g.vcf.gz -O ./variants/cohort.g.vcf.gz
14:25:52.849 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
SLF4J(W): Class path contains multiple SLF4J providers.
SLF4J(W): Found provider [org.apache.logging.slf4j.SLF4JServiceProvider@559991f5]
SLF4J(W): Found provider [ch.qos.logback.classic.spi.LogbackServiceProvider@34c76167]
SLF4J(W): See https://www.slf4j.org/codes.html#multiple_bindings for an

Vamos transformar arquivos de variantes no formato GVCF (Genomic Variant Call Format) em um VCF final que contém as variantes genotipadas.

In [ ]:
#  Genotipagem de variantes
!gatk GenotypeGVCFs -R /content/refGen/chr17_GRCh38.fasta -V ./variants/cohort.g.vcf.gz -O ./variants/cohort.vcf.gz


Using GATK jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar GenotypeGVCFs -R /content/refGen/chr17_GRCh38.fasta -V ./variants/cohort.g.vcf.gz -O ./variants/cohort.vcf.gz
14:27:13.554 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
SLF4J(W): Class path contains multiple SLF4J providers.
SLF4J(W): Found provider [org.apache.logging.slf4j.SLF4JServiceProvider@7364f68]
SLF4J(W): Found provider [ch.qos.logback.classic.spi.LogbackServiceProvider@55a0f011]
SLF4J(W): See https://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J(I): Actual provider is of type [org.apache.logging.slf4j.SLF4J

Agora vamos aplicar alguns filtros de qualidade das chamadas de variantes.

Abaixo algumas legendas:

*   QD (QualByDepth): é calculado como a razão entre a qualidade de chamada de variante (QUAL) e a profundidade de cobertura no local da variante (DP).
*   FS (Fisher Strand Bias): é uma métrica que mede a presença de viés de strand em torno de uma chamada de variante. Ocorre quando as variantes são detectadas preferencialmente em um dos strands (fitas) do DNA durante o sequenciamento
*   MQ (Mapping Quality): representa a qualidade de mapeamento das leituras (reads) que suportam a variante.





In [ ]:
# Aplicando alguns Filtros de qualidade
!gatk VariantFiltration -R /content/refGen/chr17_GRCh38.fasta -V ./variants/cohort.vcf.gz -O ./variants/cohort_filtered.vcf.gz --filter-expression "QD < 2.0 || FS > 60.0 || MQ < 40.0" --filter-name "my_snp_filter"

Using GATK jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar VariantFiltration -R /content/refGen/chr17_GRCh38.fasta -V ./variants/cohort.vcf.gz -O ./variants/cohort_filtered.vcf.gz --filter-expression QD < 2.0 || FS > 60.0 || MQ < 40.0 --filter-name my_snp_filter
14:27:33.654 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/usr/local/share/gatk4-4.6.1.0-0/gatk-package-4.6.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
SLF4J(W): Class path contains multiple SLF4J providers.
SLF4J(W): Found provider [org.apache.logging.slf4j.SLF4JServiceProvider@7a64cb0c]
SLF4J(W): Found provider [ch.qos.logback.classic.spi.LogbackServiceProvider@785ed99c]
SLF4J(W): See https://www.slf4j.org/codes.html#multiple_bi

Verificando o arquivo com as variantes chamadas.

Algumas informações importantes:

* GT (Genotype): Representa o genótipo da amostra para aquela posição. Por exemplo, 0/1 indica que a amostra é heterozigota (uma cópia de cada alelo), enquanto 1/1 indica homozigoto para o alelo alternativo.

* AD (Allelic Depth): Mostra as contagens de leitura para os alelos de referência e alternativo. Por exemplo, AD=10,5 significa que foram observadas 10 leituras para o alelo de referência e 5 leituras para o alelo alternativo.

* DP (Depth of Coverage): Representa o número total de leituras que cobrem a posição variante, ou seja, a soma dos valores de AD. No exemplo anterior, seria 15 (10 leituras de referência + 5 leituras alternativos).

* GQ (Genotype Quality): Indica a confiança da chamada do genótipo. Geralmente, quanto maior o valor, maior a confiabilidade do genótipo chamado.

* PL (Phred-scaled Likelihoods): Indica as probabilidades relativas de cada genótipo possível (homozigoto referência, heterozigoto, homozigoto alternativo) em uma escala Phred. Quanto menor o valor, mais provável é o genótipo. Por exemplo, PL=0,50,100 indica que o genótipo homozigoto referência é o mais provável (PL=0).

In [ ]:
!zcat ./variants/cohort_filtered.vcf.gz

##fileformat=VCFv4.2
##ALT=<ID=NON_REF,Description="Represents any possible alternative allele not already represented at this location by REF and ALT">
##FILTER=<ID=LowQual,Description="Low quality">
##FILTER=<ID=my_snp_filter,Description="QD < 2.0 || FS > 60.0 || MQ < 40.0">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=MIN_DP,Number=1,Type=Integer,Description="Minimum DP observed within the GVCF block">
##FORMAT=<ID=PGT,Number=1,Type=String,Description="Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another; will always be heterozygous and is not intended to describe called alleles">
##FOR

#Anotação de variantes

Existem várias bases de dados com informações de variantes conhecidas. Temos também ferramentas como o ANNOVAR que é capaz de baixar essas bases e fazer anotações nos nossos arquivos de variantes, adicionando informações importantes. Aqui usaremos o bcftools, que é uma ferramenta para manipulação e análise de VCFs, podendo adicionar anotações baseadas em uma base de dados.

Para mais informações:

* [bcftools](https://github.com/samtools/bcftools)
* [ANNOVAR](https://annovar.openbioinformatics.org/en/latest/)


In [ ]:
# Criando ambiente mamba para o BCFTOOLS e instalando a aplicação
!mamba create -n bcftools
!mamba activate bcftools
!mamba install -c bioconda bcftools --quiet


Looking for: []

Preparing transaction: - done
Verifying transaction: | / - \ | / - \ | / - \ | / - \ | / done
Executing transaction: \ done

To activate this environment, use

     $ mamba activate bcftools

To deactivate an active environment, use

     $ mamba deactivate

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Agora faremos a anotação com o bcftools e nossa minibase do Clinvar.

In [ ]:
!echo -e "\nCriando a pasta annotated"
!mkdir annotated
# Anotar o VCF com os rsIDs usando o arquivo dbSNP para hg38
!echo -e "\nAnotando o arquivo cohort_filtered.vcf.gz"
# Anotação com o bcftools e a minibase
!bcftools annotate -a ./brca1/clinvar_mini.vcf.gz -c ID,INFO ./variants/cohort_filtered.vcf.gz -o ./annotated/cohort_annotated.vcf



Criando a pasta annotated

Anotando o arquivo cohort_filtered.vcf.gz


In [ ]:
!cat ./annotated/cohort_annotated.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##ALT=<ID=NON_REF,Description="Represents any possible alternative allele not already represented at this location by REF and ALT">
##FILTER=<ID=LowQual,Description="Low quality">
##FILTER=<ID=my_snp_filter,Description="QD < 2.0 || FS > 60.0 || MQ < 40.0">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=MIN_DP,Number=1,Type=Integer,Description="Minimum DP observed within the GVCF block">
##FORMAT=<ID=PGT,Number=1,Type=String,Description="Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another; will always be heterozygous an

##OBRIGADO!